In [1]:
## imports
import sys
import os
import numpy as np
import xarray as xr
import cmocean
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import gridspec
import cartopy
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import eofs
from eofs.xarray import Eof
import cftime
import nc_time_axis

import mtspec

from pylab import *

import datetime
import time
import glob, os
import math
import matplotlib.colors as colors

## E280

In [21]:
experiment = 'E280' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/temp/'
filename_data= 'new_nemo_bq637o_1m_*-*_grid-T.nc'
files = glob.glob(folder+filename_data)
files.sort()

xr.open_dataset(files[0])

<xarray.Dataset>
Dimensions:       (bnds: 2, deptht: 75, nvertex: 4, time_counter: 1, x: 362, y: 332)
Coordinates:
  * time_counter  (time_counter) float64 0.0
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
  * deptht        (deptht) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
Dimensions without coordinates: bnds, nvertex, x, y
Data variables:
    nav_lon_bnds  (y, x, nvertex) float32 ...
    nav_lat_bnds  (y, x, nvertex) float32 ...
    cell_area     (y, x) float32 ...
    deptht_bnds   (deptht, bnds) float32 ...
    thetao        (time_counter, deptht, y, x) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.8 (https://mpimet.mpg.de...
    Conventions:  CF-1.5
    history:      Mon Oct 11 14:11:14 2021: cdo -selname,thetao /gws/nopw/j04...
    name:         bq637o_1m_19500101_19501230
    description:  ocean T grid variables
    title:        ocean T grid variables
    production:   An IPSL model
    timeStamp:    2020-Jul-20 12:27:28 UTC
    CDO:          Climate Data Operators version 1.9.8 (https://mpimet.mpg.de...

In [49]:
## Potential temperature
#Monthtly to yearly files
year = 1950

for j in range(100):
    ds = xr.open_dataset(files[j*12])
    thetao = ds.thetao[0]
    thetao = thetao.assign_coords(time=int(0))
    thetao = thetao.expand_dims('time')
    for i in range(1,12):
        ds1 = xr.open_dataset(files[j*12+i])
        thetao1 = ds1.thetao[0]
        thetao1 = thetao1.assign_coords(time=int(i))
        thetao1 = thetao1.expand_dims('time')
        thetao = xr.merge([thetao, thetao1])
    thetao = thetao.mean(dim='time')
    thetao = thetao.assign_coords(time=int(year))
    thetao = thetao.expand_dims('time')

    xr.Dataset.to_netcdf(thetao, path = folder+'Yearlyfiles/thetao_'+str(year)+'.nc')
    year = year+1

In [22]:
#Yearly files to one file
year1 = 1950
ds = xr.open_dataset(folder+'Yearlyfiles/thetao_'+str(year1)+'.nc')
thetao = ds.thetao
for i in range(1,100):
    ds1 = xr.open_dataset(folder+'Yearlyfiles/thetao_'+str(year1+i)+'.nc')
    thetao1 = ds1.thetao
    thetao = xr.merge([thetao, thetao1])

thetaoE280 = thetao    
xr.Dataset.to_netcdf(thetaoE280, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/thetao_annual_100yr.nc')

thetaoE280_100yr = thetaoE280.mean(dim='time')
xr.Dataset.to_netcdf(thetaoE280_100yr, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/thetao_100yr.nc')

In [54]:
## Meridional velocity
#Monthly files to yearly files

experiment = 'E280' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/v/'
filename_data= 'new_nemo_bq637o_1m_*-*_grid-V.nc'
files = glob.glob(folder+filename_data)
files.sort()

year = 1950

for j in range(100):
    ds = xr.open_dataset(files[j*12])
    vo = ds.vo[0]
    vo = vo.assign_coords(time=int(0))
    vo = vo.expand_dims('time')
    for i in range(1,12):
        ds1 = xr.open_dataset(files[j*12+i])
        vo1 = ds1.vo[0]
        vo1 = vo1.assign_coords(time=int(i))
        vo1 = vo1.expand_dims('time')
        vo = xr.merge([vo, vo1])
    vo = vo.mean(dim='time')
    vo = vo.assign_coords(time=int(year))
    vo = vo.expand_dims('time')

    xr.Dataset.to_netcdf(vo, path = folder+'Yearlyfiles/vo_'+str(year)+'.nc')
    year = year+1

In [24]:
#Yearly files to one file
year1 = 1950
experiment = 'E280' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/v/'
ds = xr.open_dataset(folder+'Yearlyfiles/vo_'+str(year1)+'.nc')
vo = ds.vo
for i in range(1,100):
    ds1 = xr.open_dataset(folder+'Yearlyfiles/vo_'+str(year1+i)+'.nc')
    vo1 = ds1.vo
    vo = xr.merge([vo, vo1])

voE280 = vo    
xr.Dataset.to_netcdf(voE280, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/vo_annual_100yr.nc')

voE280_100yr = voE280.mean(dim='time')
xr.Dataset.to_netcdf(voE280_100yr, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/vo_100yr.nc')

In [7]:
## Salinity
#Monthly files to yearly files
experiment = 'E280' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/sal/'
filename_data= 'new_nemo_bq637o_1m_*-*_grid-T.nc'
files = glob.glob(folder+filename_data)
files.sort()

year = 1950

for j in range(100):
    ds = xr.open_dataset(files[j*12])
    so = ds.so[0]
    so = so.assign_coords(time=int(0))
    so = so.expand_dims('time')
    for i in range(1,12):
        ds1 = xr.open_dataset(files[j*12+i])
        so1 = ds1.so[0]
        so1 = so1.assign_coords(time=int(i))
        so1 = so1.expand_dims('time')
        so = xr.merge([so, so1])
    so = so.mean(dim='time')
    so = so.assign_coords(time=int(year))
    so = so.expand_dims('time')

    xr.Dataset.to_netcdf(so, path = folder+'Yearlyfiles/so_'+str(year)+'.nc')
    year = year+1

In [8]:
#Yearly files to one file
year1 = 1950
experiment = 'E280' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/sal/'
ds = xr.open_dataset(folder+'Yearlyfiles/so_'+str(year1)+'.nc')
so = ds.so
for i in range(1,100):
    ds1 = xr.open_dataset(folder+'Yearlyfiles/so_'+str(year1+i)+'.nc')
    so1 = ds1.so
    so = xr.merge([so, so1])

soE280 = so    
xr.Dataset.to_netcdf(soE280, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/so_annual_100yr.nc')

soE280_100yr = soE280.mean(dim='time')
xr.Dataset.to_netcdf(soE280_100yr, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/so_100yr.nc')

## Eoi400 processing

In [18]:
## Potential temperature

#Monthly to yearly files
experiment = 'Eoi400' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/temp/'
filename_data= '*-*_grid-T.nc'
files = glob.glob(folder+filename_data)
files.sort()

year = 2394

for j in range(40):
    ds = xr.open_dataset(files[(j+60)*12])
    thetao = ds.thetao[0]
    thetao = thetao.assign_coords(time=int(0))
    thetao = thetao.expand_dims('time')
    for i in range(1,12):
        ds1 = xr.open_dataset(files[(j+60)*12+i])
        thetao1 = ds1.thetao[0]
        thetao1 = thetao1.assign_coords(time=int(i))
        thetao1 = thetao1.expand_dims('time')
        thetao = xr.merge([thetao, thetao1])
    thetao = thetao.mean(dim='time')
    thetao = thetao.assign_coords(time=int(year))
    thetao = thetao.expand_dims('time')

    xr.Dataset.to_netcdf(thetao, path = folder+'Yearlyfiles/thetao_'+str(year)+'.nc')
    year = year+1

In [25]:
#Yearly to one file
experiment = 'Eoi400' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/temp/'
year1 = 2334
ds = xr.open_dataset(folder+'Yearlyfiles/thetao_'+str(year1)+'.nc')
thetao = ds.thetao
for i in range(1,100):
    ds1 = xr.open_dataset(folder+'Yearlyfiles/thetao_'+str(year1+i)+'.nc')
    thetao1 = ds1.thetao
    thetao = xr.merge([thetao, thetao1])

thetaoE280 = thetao    
xr.Dataset.to_netcdf(thetaoE280, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/thetao_annual_100yr.nc')

thetaoE280_100yr = thetaoE280.mean(dim='time')
xr.Dataset.to_netcdf(thetaoE280_100yr, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/thetao_100yr.nc')

In [19]:
## Meridional velocity

#Monthly to yearly files
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/v/'
filename_data= '*-*_grid-V.nc'
files = glob.glob(folder+filename_data)
files.sort()

year = 2334

for j in range(100):
    ds = xr.open_dataset(files[j*12])
    vo = ds.vo[0]
    vo = vo.assign_coords(time=int(0))
    vo = vo.expand_dims('time')
    for i in range(1,12):
        ds1 = xr.open_dataset(files[j*12+i])
        vo1 = ds1.vo[0]
        vo1 = vo1.assign_coords(time=int(i))
        vo1 = vo1.expand_dims('time')
        vo = xr.merge([vo, vo1])
    vo = vo.mean(dim='time')
    vo = vo.assign_coords(time=int(year))
    vo = vo.expand_dims('time')

    xr.Dataset.to_netcdf(vo, path = folder+'Yearlyfiles/vo_'+str(year)+'.nc')
    year = year+1

In [26]:
#Yearly files to one file
experiment = 'Eoi400' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/v/'
year1 = 2334
ds = xr.open_dataset(folder+'Yearlyfiles/vo_'+str(year1)+'.nc')
vo = ds.vo
for i in range(1,100):
    ds1 = xr.open_dataset(folder+'Yearlyfiles/vo_'+str(year1+i)+'.nc')
    vo1 = ds1.vo
    vo = xr.merge([vo, vo1])

voE280 = vo    
xr.Dataset.to_netcdf(voE280, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/vo_annual_100yr.nc')

voE280_100yr = voE280.mean(dim='time')
xr.Dataset.to_netcdf(voE280_100yr, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/vo_100yr.nc')

In [9]:
## Salinity

#Monthly to yearly files
experiment = 'Eoi400' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/sal/'
filename_data= '*-*_grid-T.nc'
files = glob.glob(folder+filename_data)
files.sort()

year = 2334

for j in range(100):
    ds = xr.open_dataset(files[j*12])
    so = ds.so[0]
    so = so.assign_coords(time=int(0))
    so = so.expand_dims('time')
    for i in range(1,12):
        ds1 = xr.open_dataset(files[j*12+i])
        so1 = ds1.so[0]
        so1 = so1.assign_coords(time=int(i))
        so1 = so1.expand_dims('time')
        so = xr.merge([so, so1])
    so = so.mean(dim='time')
    so = so.assign_coords(time=int(year))
    so = so.expand_dims('time')

    xr.Dataset.to_netcdf(so, path = folder+'Yearlyfiles/so_'+str(year)+'.nc')
    year = year+1

In [2]:
#Yearly files to one file
year1 = 2334
experiment = 'Eoi400' 
folder = '/Volumes/External/HadGEM3 data/'+experiment+'/native_grid/sal/'
ds = xr.open_dataset(folder+'Yearlyfiles/so_'+str(year1)+'.nc')
so = ds.so
for i in range(1,100):
    ds1 = xr.open_dataset(folder+'Yearlyfiles/so_'+str(year1+i)+'.nc')
    so1 = ds1.so
    so = xr.merge([so, so1])

soEoi400 = so    
xr.Dataset.to_netcdf(soEoi400, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/so_annual_100yr.nc')

soEoi400_100yr = soEoi400.mean(dim='time')
xr.Dataset.to_netcdf(soEoi400_100yr, path = '/Users/6497241/surfdrive/Documents/PlioMIP2-OHT/Data/Raw_data/HadGEM3/'+experiment+'/so_100yr.nc')